In [ ]:
# This cell imports all required libraries and setups some helpers
import micropip
await micropip.install(["ipywidgets", "pandas", "plotly==5.1.0"])

from IPython.display import display, clear_output, JSON
from ipywidgets import interact, widgets

import js
import json
import os
import pandas as pd
import plotly.express as px


class Camunda:
    @staticmethod
    def get(path):
        request = js.XMLHttpRequest.new()
        request.open("GET", f"{os.environ['CAMUNDA_ENGINE_API']}{path}", False)
        request.send(None)
        assert request.status in [200, 204], request.responseText
        return json.loads(request.responseText or 'null')

At first, let's test our helpers to fetch past processes and display them with pandas DataFrame:

In [ ]:
pd.DataFrame(Camunda.get("/history/process-instance"))

Next, we could fetch the data into a variable and select only the columns we are interested in.

In [ ]:
df = pd.DataFrame(Camunda.get("/history/process-instance"))
df = df[["processDefinitionKey", "durationInMillis"]]
df

Finally, we could do some plotting

In [ ]:
px.box(df, x="processDefinitionKey", y="durationInMillis", color="processDefinitionKey")

Or do everything in a single line:

In [ ]:
px.box(pd.DataFrame(Camunda.get("/history/process-instance"))[["processDefinitionKey", "durationInMillis"]], x="processDefinitionKey", y="durationInMillis", color="processDefinitionKey")

Jupyter Widgets could be used to interactively choose the data:

In [ ]:
@interact(key=widgets.RadioButtons(
    options=['traffic-data-loader', 'traffic-data-loader-parallel'],
    value='traffic-data-loader',
    disabled=False,
))
def plot(key):
    display(px.box(df[df.processDefinitionKey == key], x="processDefinitionKey", y="durationInMillis", color="processDefinitionKey"))